In [14]:
import boto3
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms.bedrock import Bedrock
from langchain.document_loaders import S3FileLoader
from bedrock import get_bedrock_client
bedrock_client = get_bedrock_client(region='us-east-1', runtime=True)

s3 = boto3.client('s3')

bucket_name = 'my-s3-doc-loader'
response = s3.list_objects_v2(Bucket=bucket_name)

file_names = []
for obj in response['Contents']:
    key = obj['Key']
    file_names.append(key)

for context_key in file_names:
    loader = S3FileLoader(bucket=bucket_name, key=context_key)
    context_content = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000)
context_texts = text_splitter.split_documents(context_content)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(documents=context_texts, embedding=embeddings)

retriever = db.as_retriever(search_type='mmr', search_kwargs={"k": 5})

template = """
Human: Answer truthfully based on the given question, fetch the answer only from the given text documents
Instruction:
1.If multiple files are there, read the all the files each and every lines accurately for to generate answer
2.If there is no text found in the text document about the asked question ,"print no result found" do not print any results if answer not found,do not search the answers from outside
3.Generate answer whatever available related to the question
4.Must complete the sentence in the result fully, do not leave results incomplete format in the end.
text:{context}
question:{question}
Assistant:"""
qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])
chain_type_kwargs = { "prompt": qa_prompt}
llm = Bedrock(model_id="anthropic.claude-v2",client=bedrock_client)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)


In [20]:
question="what is Arduino IDE?"
result = qa.run(question)
print(result)

 Based on the given text documents, here is the relevant information about Arduino IDE:

Arduino IDE (Integrated Development Environment) is a cross-platform application written in Java that is used to write and upload programs to Arduino compatible boards. It includes a code editor, compiler, and uploader that allows users to write code in languages like C and C++ and upload it to an Arduino board. 

Some key features of Arduino IDE:

- It supports C and C++ programming languages.
- It has a built-in code editor with features like text cutting and pasting, automatic indentation, brace matching and syntax highlighting. 
- It compiles and uploads programs to the board with a single click.
- It includes a Serial Monitor that allows sending and receiving text data to and from the Arduino board over USB.
- It is compatible with all Arduino boards like Uno, Mega, Nano, etc.
- It can be extended through third-party plug-ins.
- It runs on Windows, Mac OS X, and Linux operating systems.

So in